In [1]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, 7, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)

print(latest_file)

graph_list/Town03\3_graph_v34.adjlist


In [2]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (621, 2364), (604, 1679)
if reverse == "y":
    start, end = end, start


In [3]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "Forward", False)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (604, 1679)
End point: (621, 2364)
Finding Path...
Get!!!
found!!!
[(604, 1679), (604, 1683), (604, 1687), (604, 1691), (605, 1695), (605, 1699), (605, 1703), (605, 1707), (605, 1711), (605, 1715), (605, 1719), (605, 1723), (605, 1727), (605, 1731), (606, 1735), (606, 1739), (606, 1743), (606, 1747), (606, 1751), (606, 1755), (606, 1759), (606, 1763), (606, 1767), (606, 1771), (607, 1775), (607, 1779), (607, 1783), (607, 1787), (607, 1791), (607, 1795), (607, 1799), (607, 1803), (607, 1807), (607, 1811), (608, 1815), (608, 1819), (608, 1823), (608, 1827), (608, 1831), (608, 1835), (608, 1839), (608, 1843), (608, 1847), (608, 1851), (609, 1855), (609, 1859), (609, 1863), (609, 1867), (609, 1871), (609, 1875), (609, 1879), (609, 1883), (609, 1887), (609, 1891), (610, 1895), (610, 1899), (610, 1903), (610, 1907), (610, 1911), (610, 1915), (610, 1919), (610, 1923), (610, 1927), (610, 1931), (611, 1935), (611, 1939), (611, 1943), (611, 1947), (611, 1951), (611, 1955), (611, 195

In [4]:
############### Quick test for other start point with the same goal point
start, end = (621, 2364), (627, 1476)

fp = nx.shortest_path(G,  start,  end)
draw_img = draw_waypoint(waypoint_map, start, end, fp)
rdp_img, _ = rdp_algorithm(draw_img, fp)
cv2.imwrite(f"test/check__{start}_{end}.png", rdp_img)
print(fp)

[(621, 2364), (621, 2360), (621, 2348), (621, 2336), (620, 2324), (620, 2311), (620, 2295), (619, 2283), (619, 2271), (619, 2259), (618, 2247), (618, 2235), (618, 2223), (617, 2211), (617, 2199), (617, 2187), (617, 2175), (616, 2163), (616, 2151), (616, 2139), (615, 2127), (615, 2115), (615, 2103), (614, 2091), (614, 2079), (614, 2067), (614, 2055), (613, 2043), (613, 2031), (613, 2017), (612, 2007), (612, 2000), (612, 1987), (612, 1975), (611, 1963), (611, 1951), (611, 1939), (610, 1927), (610, 1915), (610, 1903), (609, 1891), (609, 1879), (609, 1867), (609, 1855), (608, 1843), (608, 1831), (608, 1819), (607, 1807), (607, 1795), (607, 1783), (606, 1771), (606, 1759), (606, 1747), (606, 1735), (605, 1723), (605, 1711), (605, 1699), (604, 1687), (604, 1679), (604, 1675), (604, 1663), (604, 1659), (603, 1647), (603, 1643), (603, 1631), (603, 1627), (603, 1615), (602, 1611), (602, 1597), (602, 1592), (603, 1578), (603, 1573), (605, 1559), (606, 1554), (608, 1540), (609, 1535), (612, 1521)